# Finetune BERT for classification

BERT is a state-of-the-art model by Google that came in 2019. In this blog, I will go step by step to finetune the BERT model for movie reviews classification(i.e positive or negative ). Here, I will be using the Pytorch framework for the coding perspective.

First, we need to install the transformers module by Hugging face in our system to make use of all the transformers model.

In [1]:
!pip install transformers

     |████████████████████████████████| 1.5MB 23.4MB/s 
     |████████████████████████████████| 2.9MB 51.5MB/s 
     |████████████████████████████████| 890kB 53.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=4b294b13312c0c0c688590e369fb5d061defb45e513b6d99490171e8300fc93a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Then, import all the packages and modules

In [2]:
import torch
import torch.nn as nn
import transformers
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm

Now we need to prepare a dataset to finetune BERT.
This is a standard method to make a dataset in PyTorch. First, we create a class inherited from the torch Dataset module. Then, In this class, we make the __getitem__ method where the main code will be implemented to prepare a dataset.

Dataset is loaded from GitHub, which consists of 2000 movie reviews with labels and converted to pandas dataframe.


BERT tokenizer.encode_plus method takes input as a pair of text sequence with additional parameters like max_length (Controls the maximum length to be used by one of the truncation/padding parameters.), pad_to_max_length (Pad to a maximum length specified with the argument max_length ) and return_attention_mask: bool (This argument indicates to the model which tokens should be attended to, and which should not).

Tokenizer.encode_plus returns a dictionary containing the encoded tokenized sequence and other additional information such as attention_marks, token_type_id, and this would be an input to the BERT model.
In line 35, from the hugging face library, the pre-trained BERT uncased tokenizer is initialized and then taken as input in dataset class.

In [18]:
class BertDataset(Dataset):
    def __init__(self,root_dir, train_csv,tokenizer,max_length):
        super(BertDataset, self).__init__()
        self.root_dir=root_dir
        self.train_csv=pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length
        
    def __len__(self):
        return len(self.train_csv)
    
    def __getitem__(self, index):
        
        text1 = self.train_csv.iloc[index,0]
        
        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(self.train_csv.iloc[index, 1], dtype=torch.long)
            }



In [19]:
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased")

In [20]:
dataset= BertDataset(r'/content/drive/MyDrive/Bert train','train.csv',tokenizer,
                     max_length=100)

Now, we have prepared our dataset and it will be a perfect time to create and initialize our BERT model.

In [23]:
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.bert_model = transformers.BertModel.from_pretrained("bert-base-uncased")
        self.out = nn.Linear(768, 1)
        
    def forward(self,ids,mask,token_type_ids):
        _,o2= self.bert_model(ids,attention_mask=mask,token_type_ids=token_type_ids, return_dict=False)
        
        out= self.out(o2)
        
        return out


loss_fn = nn.BCEWithLogitsLoss()


we use BCEWithLogitsLoss which combines a Sigmoid layer and the BCELoss in one single class because this version is more numerically stable than using a plain Sigmoid followed by a BCELoss.

We have prepared the dataset and also we have defined our BERT model. Now, it’s time to train our model.

In [24]:
def finetune(epochs,dataloader,model,loss_fn,optimizer):
    model.train()
    for  epoch in range(epochs):
        print(epoch)
        
        loop=tqdm(enumerate(dataloader),leave=False,total=len(dataloader))
        for batch, dl in loop:
            ids=dl['ids']
            token_type_ids=dl['token_type_ids']
            mask= dl['mask']
            label=dl['target']
            label = label.unsqueeze(1)
            output=model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids)
            label = label.type_as(output)

            loss=loss_fn(output,label)
            optimizer.zero_grad()
            loss.backward()
            
            optimizer.step()
            pred = np.where(output >= 0, 1, 0)


            num_correct = sum(1 for a, b in zip(pred, label) if a[0] == b[0])
            num_samples = pred.shape[0]
            accuracy = num_correct/num_samples
            print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
            # Show progress while training
            loop.set_description(f'Epoch={epoch}/{epochs}')
            loop.set_postfix(loss=loss.item(),acc=accuracy)

    return model



In [25]:
model=BERT()

In [26]:
model.named_parameters

<bound method Module.named_parameters of BERT(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768

Also, now we have two situations:

First, we do not retrain our pre-trained BERT and train only the last linear dense layer.

Second, we train our complete BERT model. For this, we don’t need to do anything initially, model parameters required_grad is set to True.

In [27]:
for param in model.bert_model.parameters():
    param.requires_grad = False

In [28]:

dataloader=DataLoader(dataset=dataset,batch_size=32)

#Initialize Optimizer
optimizer= optim.Adam(model.parameters(),lr= 0.00001)
model=finetune(2, dataloader, model, loss_fn, optimizer)

  0%|          | 0/217 [00:00<?, ?it/s]

0


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Epoch=0/2:   0%|          | 1/217 [00:07<28:24,  7.89s/it, acc=0.469, loss=0.749]

Got 15 / 32 with accuracy 46.88


Epoch=0/2:   1%|          | 2/217 [00:15<28:16,  7.89s/it, acc=0.75, loss=0.582]

...

Epoch=1/2:  99%|█████████▉| 215/217 [27:29<00:15,  7.69s/it, acc=0.469, loss=0.69]

Got 15 / 32 with accuracy 46.88


Epoch=1/2: 100%|█████████▉| 216/217 [27:37<00:07,  7.74s/it, acc=0.625, loss=0.678]

Got 20 / 32 with accuracy 62.50


Got 3 / 8 with accuracy 37.50
